In [1]:
import numpy as np
import pandas as pd
from sklearn import ensemble
from sklearn import metrics
from sklearn import model_selection
import optuna
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
import warnings

# read the training data
df = pd.read_csv("loan_train.csv")

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   ID                  4000 non-null   int64
 1   Age                 4000 non-null   int64
 2   Experience          4000 non-null   int64
 3   ZIP Code            4000 non-null   int64
 4   Family              4000 non-null   int64
 5   Education           4000 non-null   int64
 6   Mortgage            4000 non-null   int64
 7   Securities Account  4000 non-null   int64
 8   CD Account          4000 non-null   int64
 9   Online              4000 non-null   int64
 10  CreditCard          4000 non-null   int64
 11  Personal Loan       4000 non-null   int64
dtypes: int64(12)
memory usage: 375.1 KB


In [3]:
df.head()

,ID,Age,Experience,ZIP Code,Family,Education,Mortgage,Securities Account,CD Account,Online,CreditCard,Personal Loan
0,3510,38,12,91330,3,3,0,0,0,0,0,0
1,1129,30,5,94025,2,2,0,0,0,0,0,1
2,1637,65,39,92122,4,3,0,0,0,0,1,0
3,3165,28,4,95136,4,1,0,0,0,1,1,0
4,3563,32,8,94596,1,3,272,1,1,1,0,1


In [4]:
# here we have training features
df.drop(["ZIP Code","ID"],axis=1,inplace=True)
X = df.drop("Personal Loan", axis=1).values
# and the targets
y = df["Personal Loan"].values

## BO for each Model

### Random Forest

In [5]:
from sklearn.ensemble import RandomForestClassifier
def objective(trial):
    
    # optuna.trial.Trial.suggest_categorical() for categorical parameters
    # optuna.trial.Trial.suggest_int() for integer parameters
    # optuna.trial.Trial.suggest_float() for floating point parameters    
    rf_bootstrap=trial.suggest_categorical("bootstrap", [True,False])
    rf_n_estimators = trial.suggest_int("n_estimators", 100, 2000, step=100)
    rf_max_depth = trial.suggest_int("max_depth", 2, 20)
    rf_max_features = trial.suggest_categorical("max_features", ["auto", "sqrt"])
    rf_criterion = trial.suggest_categorical("criterion", ["gini", "entropy"])
    rf_min_samples_split = trial.suggest_int("min_samples_split", 2, 40)
    rf_min_samples_leaf = trial.suggest_int("min_samples_leaf", 2, 40)

    classifier_obj = ensemble.RandomForestClassifier(
        bootstrap=rf_bootstrap,
        n_estimators=rf_n_estimators, 
        max_depth=rf_max_depth,
        criterion=rf_criterion,
        n_jobs=-1, 
        min_samples_leaf=rf_min_samples_leaf,
        min_samples_split=rf_min_samples_split,
        random_state=0        )
    
    score = model_selection.cross_val_score(classifier_obj, X, y, cv=5, n_jobs=-1,scoring="roc_auc")
    rocauc = score.mean()
    return rocauc

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)
print(study.best_trial)

[I 2021-10-12 11:33:01,970] A new study created in memory with name: no-name-42c3d19a-7e9b-4b1f-ac98-ad35fc1be895
[I 2021-10-12 11:33:15,559] Trial 0 finished with value: 0.7985708873211829 and parameters: {'bootstrap': True, 'n_estimators': 600, 'max_depth': 7, 'max_features': 'sqrt', 'criterion': 'entropy', 'min_samples_split': 13, 'min_samples_leaf': 28}. Best is trial 0 with value: 0.7985708873211829.
[I 2021-10-12 11:33:25,366] Trial 1 finished with value: 0.7964820471735222 and parameters: {'bootstrap': True, 'n_estimators': 400, 'max_depth': 12, 'max_features': 'auto', 'criterion': 'gini', 'min_samples_split': 32, 'min_samples_leaf': 23}. Best is trial 0 with value: 0.7985708873211829.
[I 2021-10-12 11:33:31,730] Trial 2 finished with value: 0.8031359592581389 and parameters: {'bootstrap': False, 'n_estimators': 500, 'max_depth': 10, 'max_features': 'sqrt', 'criterion': 'gini', 'min_samples_split': 38, 'min_samples_leaf': 36}. Best is trial 2 with value: 0.8031359592581389.
[I 2

[I 2021-10-12 11:37:38,140] Trial 27 finished with value: 0.8133056551920701 and parameters: {'bootstrap': True, 'n_estimators': 800, 'max_depth': 10, 'max_features': 'sqrt', 'criterion': 'entropy', 'min_samples_split': 23, 'min_samples_leaf': 4}. Best is trial 23 with value: 0.8225382900189759.
[I 2021-10-12 11:37:47,695] Trial 28 finished with value: 0.8147988508122342 and parameters: {'bootstrap': False, 'n_estimators': 1200, 'max_depth': 6, 'max_features': 'sqrt', 'criterion': 'entropy', 'min_samples_split': 30, 'min_samples_leaf': 20}. Best is trial 23 with value: 0.8225382900189759.
[I 2021-10-12 11:37:55,681] Trial 29 finished with value: 0.8119778517114075 and parameters: {'bootstrap': True, 'n_estimators': 600, 'max_depth': 7, 'max_features': 'sqrt', 'criterion': 'entropy', 'min_samples_split': 27, 'min_samples_leaf': 8}. Best is trial 23 with value: 0.8225382900189759.
[I 2021-10-12 11:38:19,691] Trial 30 finished with value: 0.8112486295822439 and parameters: {'bootstrap': F

[I 2021-10-12 11:42:15,025] Trial 54 finished with value: 0.8222257836352146 and parameters: {'bootstrap': False, 'n_estimators': 700, 'max_depth': 8, 'max_features': 'sqrt', 'criterion': 'entropy', 'min_samples_split': 16, 'min_samples_leaf': 3}. Best is trial 53 with value: 0.8227303024267594.
[I 2021-10-12 11:42:21,342] Trial 55 finished with value: 0.8123416355802465 and parameters: {'bootstrap': False, 'n_estimators': 600, 'max_depth': 5, 'max_features': 'sqrt', 'criterion': 'entropy', 'min_samples_split': 15, 'min_samples_leaf': 3}. Best is trial 53 with value: 0.8227303024267594.
[I 2021-10-12 11:42:31,228] Trial 56 finished with value: 0.811012766267712 and parameters: {'bootstrap': False, 'n_estimators': 700, 'max_depth': 16, 'max_features': 'sqrt', 'criterion': 'entropy', 'min_samples_split': 20, 'min_samples_leaf': 3}. Best is trial 53 with value: 0.8227303024267594.
[I 2021-10-12 11:42:39,799] Trial 57 finished with value: 0.8061089399967779 and parameters: {'bootstrap': Fa

[I 2021-10-12 11:45:11,023] Trial 81 finished with value: 0.822085801509929 and parameters: {'bootstrap': False, 'n_estimators': 700, 'max_depth': 7, 'max_features': 'sqrt', 'criterion': 'entropy', 'min_samples_split': 15, 'min_samples_leaf': 4}. Best is trial 65 with value: 0.8230562294322098.
[I 2021-10-12 11:45:18,526] Trial 82 finished with value: 0.8217679447800984 and parameters: {'bootstrap': False, 'n_estimators': 700, 'max_depth': 7, 'max_features': 'sqrt', 'criterion': 'entropy', 'min_samples_split': 19, 'min_samples_leaf': 5}. Best is trial 65 with value: 0.8230562294322098.
[I 2021-10-12 11:45:26,313] Trial 83 finished with value: 0.8192961185532589 and parameters: {'bootstrap': False, 'n_estimators': 800, 'max_depth': 6, 'max_features': 'sqrt', 'criterion': 'entropy', 'min_samples_split': 18, 'min_samples_leaf': 3}. Best is trial 65 with value: 0.8230562294322098.
[I 2021-10-12 11:45:37,348] Trial 84 finished with value: 0.8238333133505771 and parameters: {'bootstrap': Fal

FrozenTrial(number=84, values=[0.8238333133505771], datetime_start=datetime.datetime(2021, 10, 12, 11, 45, 26, 317260), datetime_complete=datetime.datetime(2021, 10, 12, 11, 45, 37, 347704), params={'bootstrap': False, 'n_estimators': 900, 'max_depth': 7, 'max_features': 'sqrt', 'criterion': 'entropy', 'min_samples_split': 15, 'min_samples_leaf': 2}, distributions={'bootstrap': CategoricalDistribution(choices=(True, False)), 'n_estimators': IntUniformDistribution(high=2000, low=100, step=100), 'max_depth': IntUniformDistribution(high=20, low=2, step=1), 'max_features': CategoricalDistribution(choices=('auto', 'sqrt')), 'criterion': CategoricalDistribution(choices=('gini', 'entropy')), 'min_samples_split': IntUniformDistribution(high=40, low=2, step=1), 'min_samples_leaf': IntUniformDistribution(high=40, low=2, step=1)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=84, state=TrialState.COMPLETE, value=None)


In [6]:
model_score={}
model_score["RandomForestClassifier"]=study.best_value
RF_model = ensemble.RandomForestClassifier(**study.best_params, n_jobs=-1)
RF_model.fit(X, y)

RandomForestClassifier(bootstrap=False, criterion='entropy', max_depth=7,
                       max_features='sqrt', min_samples_leaf=2,
                       min_samples_split=15, n_estimators=900, n_jobs=-1)

### Logistic Regression

In [20]:
from sklearn.linear_model import LogisticRegression
def objective(trial):
       
    lr_penalty = trial.suggest_categorical("penalty", ["l1","l2", "elasticnet"])
    lr_C = trial.suggest_categorical("C", [0.01,0.1,1,5,10,20])
    lr_fit_intercept=trial.suggest_categorical("fit_intercept",[True,False])
    lr_solver=trial.suggest_categorical("solver",["newton-cg","lbfgs","liblinear","sag","saga"])

    classifier_obj = LogisticRegression(
        penalty=lr_penalty, 
        C=lr_C,
        n_jobs=-1, 
        fit_intercept=lr_fit_intercept,
        solver=lr_solver,
        random_state=0        
    )

    score = model_selection.cross_val_score(classifier_obj, X, y, cv=5, n_jobs=-1,scoring="roc_auc")
    rocauc = score.mean()
    return rocauc


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)
print(study.best_trial)

[I 2021-10-12 11:56:53,379] A new study created in memory with name: no-name-3964bedf-0553-4ad2-a490-1349dc524496
[W 2021-10-12 11:56:53,412] Trial 0 failed, because the objective function returned nan.
[I 2021-10-12 11:56:53,613] Trial 1 finished with value: 0.7017249463299183 and parameters: {'penalty': 'l2', 'C': 0.01, 'fit_intercept': False, 'solver': 'lbfgs'}. Best is trial 1 with value: 0.7017249463299183.
[W 2021-10-12 11:56:53,641] Trial 2 failed, because the objective function returned nan.
[I 2021-10-12 11:56:53,865] Trial 3 finished with value: 0.7683575916458453 and parameters: {'penalty': 'l2', 'C': 20, 'fit_intercept': False, 'solver': 'newton-cg'}. Best is trial 3 with value: 0.7683575916458453.
[W 2021-10-12 11:56:53,894] Trial 4 failed, because the objective function returned nan.
[W 2021-10-12 11:56:53,924] Trial 5 failed, because the objective function returned nan.
[W 2021-10-12 11:56:53,955] Trial 6 failed, because the objective function returned nan.
[W 2021-10-12

[W 2021-10-12 11:56:58,191] Trial 67 failed, because the objective function returned nan.
[W 2021-10-12 11:56:58,223] Trial 68 failed, because the objective function returned nan.
[W 2021-10-12 11:56:58,262] Trial 69 failed, because the objective function returned nan.
[W 2021-10-12 11:56:58,311] Trial 70 failed, because the objective function returned nan.
[W 2021-10-12 11:56:58,352] Trial 71 failed, because the objective function returned nan.
[W 2021-10-12 11:56:58,396] Trial 72 failed, because the objective function returned nan.
[W 2021-10-12 11:56:58,445] Trial 73 failed, because the objective function returned nan.
[W 2021-10-12 11:56:58,481] Trial 74 failed, because the objective function returned nan.
[W 2021-10-12 11:56:58,517] Trial 75 failed, because the objective function returned nan.
[W 2021-10-12 11:56:58,555] Trial 76 failed, because the objective function returned nan.
[W 2021-10-12 11:56:58,599] Trial 77 failed, because the objective function returned nan.
[W 2021-10

FrozenTrial(number=11, values=[0.7683645232582383], datetime_start=datetime.datetime(2021, 10, 12, 11, 56, 54, 67520), datetime_complete=datetime.datetime(2021, 10, 12, 11, 56, 54, 319857), params={'penalty': 'l2', 'C': 10, 'fit_intercept': False, 'solver': 'newton-cg'}, distributions={'penalty': CategoricalDistribution(choices=('l1', 'l2', 'elasticnet')), 'C': CategoricalDistribution(choices=(0.01, 0.1, 1, 5, 10, 20)), 'fit_intercept': CategoricalDistribution(choices=(True, False)), 'solver': CategoricalDistribution(choices=('newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'))}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=11, state=TrialState.COMPLETE, value=None)


In [21]:
model_score["LogisticRegression"]=study.best_value
LR_model = LogisticRegression(**study.best_params, n_jobs=-1)
LR_model.fit(X, y)

LogisticRegression(C=10, fit_intercept=False, n_jobs=-1, solver='newton-cg')

### K-NN

In [22]:
from sklearn.neighbors import KNeighborsClassifier
def objective(trial):
     
    knn_n_neighbors = trial.suggest_int("n_neighbors", 1, 300, step=5)
    knn_weights = trial.suggest_categorical("weights", ["uniform", "distance"])
    knn_metric = trial.suggest_categorical("metric", ['euclidean', 'manhattan', 'minkowski'])
    knn_algorithm=trial.suggest_categorical("algorithm", ['auto','ball_tree','kd_tree','brute'])

    classifier_obj = KNeighborsClassifier(
        n_neighbors=knn_n_neighbors, 
        weights=knn_weights,
        metric=knn_metric,
        algorithm=knn_algorithm,
        n_jobs=-1        )

    score = model_selection.cross_val_score(classifier_obj, X, y, cv=5, n_jobs=-1,scoring="roc_auc")
    rocauc = score.mean()
    return rocauc


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)
print(study.best_trial)

[I 2021-10-12 11:57:11,153] A new study created in memory with name: no-name-2f846989-df36-451f-9a01-ddb3c08786c3
C:\Users\User\anaconda3\envs\notebook\lib\site-packages\optuna\distributions.py:560: UserWarning: The distribution is specified by [1, 300] and step=5, but the range is not divisible by `step`. It will be replaced by [1, 296].
  warnings.warn(
[I 2021-10-12 11:57:11,983] Trial 0 finished with value: 0.6239334455927306 and parameters: {'n_neighbors': 236, 'weights': 'uniform', 'metric': 'euclidean', 'algorithm': 'kd_tree'}. Best is trial 0 with value: 0.6239334455927306.
C:\Users\User\anaconda3\envs\notebook\lib\site-packages\optuna\distributions.py:560: UserWarning: The distribution is specified by [1, 300] and step=5, but the range is not divisible by `step`. It will be replaced by [1, 296].
  warnings.warn(
[I 2021-10-12 11:57:13,046] Trial 1 finished with value: 0.6466539124567714 and parameters: {'n_neighbors': 266, 'weights': 'uniform', 'metric': 'manhattan', 'algorith

C:\Users\User\anaconda3\envs\notebook\lib\site-packages\optuna\distributions.py:560: UserWarning: The distribution is specified by [1, 300] and step=5, but the range is not divisible by `step`. It will be replaced by [1, 296].
  warnings.warn(
[I 2021-10-12 11:57:19,485] Trial 17 finished with value: 0.6836206223419754 and parameters: {'n_neighbors': 126, 'weights': 'distance', 'metric': 'manhattan', 'algorithm': 'auto'}. Best is trial 17 with value: 0.6836206223419754.
C:\Users\User\anaconda3\envs\notebook\lib\site-packages\optuna\distributions.py:560: UserWarning: The distribution is specified by [1, 300] and step=5, but the range is not divisible by `step`. It will be replaced by [1, 296].
  warnings.warn(
[I 2021-10-12 11:57:19,790] Trial 18 finished with value: 0.6686034329212892 and parameters: {'n_neighbors': 126, 'weights': 'uniform', 'metric': 'manhattan', 'algorithm': 'auto'}. Best is trial 17 with value: 0.6836206223419754.
C:\Users\User\anaconda3\envs\notebook\lib\site-pack

C:\Users\User\anaconda3\envs\notebook\lib\site-packages\optuna\distributions.py:560: UserWarning: The distribution is specified by [1, 300] and step=5, but the range is not divisible by `step`. It will be replaced by [1, 296].
  warnings.warn(
[I 2021-10-12 11:57:25,384] Trial 34 finished with value: 0.6805452291619549 and parameters: {'n_neighbors': 106, 'weights': 'distance', 'metric': 'manhattan', 'algorithm': 'ball_tree'}. Best is trial 17 with value: 0.6836206223419754.
C:\Users\User\anaconda3\envs\notebook\lib\site-packages\optuna\distributions.py:560: UserWarning: The distribution is specified by [1, 300] and step=5, but the range is not divisible by `step`. It will be replaced by [1, 296].
  warnings.warn(
[I 2021-10-12 11:57:25,682] Trial 35 finished with value: 0.673793806045768 and parameters: {'n_neighbors': 86, 'weights': 'distance', 'metric': 'manhattan', 'algorithm': 'auto'}. Best is trial 17 with value: 0.6836206223419754.
C:\Users\User\anaconda3\envs\notebook\lib\site-

C:\Users\User\anaconda3\envs\notebook\lib\site-packages\optuna\distributions.py:560: UserWarning: The distribution is specified by [1, 300] and step=5, but the range is not divisible by `step`. It will be replaced by [1, 296].
  warnings.warn(
[I 2021-10-12 11:57:31,189] Trial 51 finished with value: 0.6826499677643396 and parameters: {'n_neighbors': 116, 'weights': 'distance', 'metric': 'manhattan', 'algorithm': 'auto'}. Best is trial 17 with value: 0.6836206223419754.
C:\Users\User\anaconda3\envs\notebook\lib\site-packages\optuna\distributions.py:560: UserWarning: The distribution is specified by [1, 300] and step=5, but the range is not divisible by `step`. It will be replaced by [1, 296].
  warnings.warn(
[I 2021-10-12 11:57:31,477] Trial 52 finished with value: 0.6799103000349398 and parameters: {'n_neighbors': 136, 'weights': 'distance', 'metric': 'manhattan', 'algorithm': 'auto'}. Best is trial 17 with value: 0.6836206223419754.
C:\Users\User\anaconda3\envs\notebook\lib\site-pac

C:\Users\User\anaconda3\envs\notebook\lib\site-packages\optuna\distributions.py:560: UserWarning: The distribution is specified by [1, 300] and step=5, but the range is not divisible by `step`. It will be replaced by [1, 296].
  warnings.warn(
[I 2021-10-12 11:57:37,381] Trial 68 finished with value: 0.6813012370377229 and parameters: {'n_neighbors': 126, 'weights': 'distance', 'metric': 'manhattan', 'algorithm': 'brute'}. Best is trial 17 with value: 0.6836206223419754.
C:\Users\User\anaconda3\envs\notebook\lib\site-packages\optuna\distributions.py:560: UserWarning: The distribution is specified by [1, 300] and step=5, but the range is not divisible by `step`. It will be replaced by [1, 296].
  warnings.warn(
[I 2021-10-12 11:57:37,703] Trial 69 finished with value: 0.6786624250978319 and parameters: {'n_neighbors': 106, 'weights': 'distance', 'metric': 'manhattan', 'algorithm': 'auto'}. Best is trial 17 with value: 0.6836206223419754.
C:\Users\User\anaconda3\envs\notebook\lib\site-pa

C:\Users\User\anaconda3\envs\notebook\lib\site-packages\optuna\distributions.py:560: UserWarning: The distribution is specified by [1, 300] and step=5, but the range is not divisible by `step`. It will be replaced by [1, 296].
  warnings.warn(
[I 2021-10-12 11:57:46,112] Trial 85 finished with value: 0.6826290058492243 and parameters: {'n_neighbors': 151, 'weights': 'distance', 'metric': 'manhattan', 'algorithm': 'brute'}. Best is trial 17 with value: 0.6836206223419754.
C:\Users\User\anaconda3\envs\notebook\lib\site-packages\optuna\distributions.py:560: UserWarning: The distribution is specified by [1, 300] and step=5, but the range is not divisible by `step`. It will be replaced by [1, 296].
  warnings.warn(
[I 2021-10-12 11:57:46,549] Trial 86 finished with value: 0.680784679983845 and parameters: {'n_neighbors': 141, 'weights': 'distance', 'metric': 'manhattan', 'algorithm': 'auto'}. Best is trial 17 with value: 0.6836206223419754.
C:\Users\User\anaconda3\envs\notebook\lib\site-pac

FrozenTrial(number=17, values=[0.6836206223419754], datetime_start=datetime.datetime(2021, 10, 12, 11, 57, 19, 222850), datetime_complete=datetime.datetime(2021, 10, 12, 11, 57, 19, 484559), params={'n_neighbors': 126, 'weights': 'distance', 'metric': 'manhattan', 'algorithm': 'auto'}, distributions={'n_neighbors': IntUniformDistribution(high=296, low=1, step=5), 'weights': CategoricalDistribution(choices=('uniform', 'distance')), 'metric': CategoricalDistribution(choices=('euclidean', 'manhattan', 'minkowski')), 'algorithm': CategoricalDistribution(choices=('auto', 'ball_tree', 'kd_tree', 'brute'))}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=17, state=TrialState.COMPLETE, value=None)


In [23]:
model_score["KNeighborsClassifier"]=study.best_value
Knn_model = KNeighborsClassifier(**study.best_params, n_jobs=-1)
Knn_model.fit(X, y)

KNeighborsClassifier(metric='manhattan', n_jobs=-1, n_neighbors=126,
                     weights='distance')

## ROC-AUC Score

In [24]:
model_list=[RF_model,LR_model,Knn_model]
for model in model_list:
    pred=model.predict_proba(X)
    print(f'ROC-AUC SCORE of {str(model)} : {roc_auc_score(y,pred[:,1])}')

ROC-AUC SCORE of RandomForestClassifier(bootstrap=False, criterion='entropy', max_depth=7,
                       max_features='sqrt', min_samples_leaf=2,
                       min_samples_split=15, n_estimators=900, n_jobs=-1) : 0.886647452295354
ROC-AUC SCORE of LogisticRegression(C=10, fit_intercept=False, n_jobs=-1, solver='newton-cg') : 0.7758720645510693
ROC-AUC SCORE of KNeighborsClassifier(metric='manhattan', n_jobs=-1, n_neighbors=126,
                     weights='distance') : 0.9991433472759955


In [25]:
print(model_score)

{'RandomForestClassifier': 0.8238333133505771, 'LogisticRegression': 0.7683645232582383, 'KNeighborsClassifier': 0.6836206223419754}
